### Dataset

In [1]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('course_lead_scoring.csv')
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


In [3]:
# Checking missing values
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [4]:
# Checking data types
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [5]:
# Checking the number of unique values to determine whether some variables are better be put as numerical or categorical
df.nunique()

lead_source                    5
industry                       7
number_of_courses_viewed      10
annual_income               1267
employment_status              4
location                       7
interaction_count             12
lead_score                   101
converted                      2
dtype: int64

In [6]:
# Defining numerical and categorical variables
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
categorical = ['lead_source', 'industry', 'employment_status', 'location']

In [7]:
# Replacing numerical missing values with 0.0
df[numerical] = df[numerical].fillna(0.0)
df[numerical]

,number_of_courses_viewed,annual_income,interaction_count,lead_score
0,1,79450.0,4,0.94
1,1,46992.0,1,0.80
2,5,78796.0,3,0.69
3,2,83843.0,1,0.87
4,3,85012.0,3,0.62
...,...,...,...,...
1457,1,0.0,4,0.53
1458,3,65259.0,2,0.24
1459,1,45688.0,3,0.02
1460,5,71016.0,0,0.25


In [8]:
df[numerical].isna().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
dtype: int64

In [9]:
# Replacing categorical missing values with 'NA'
df[categorical] = df[categorical].fillna('NA')
df[categorical]

,lead_source,industry,employment_status,location
0,paid_ads,NA,unemployed,south_america
1,social_media,retail,employed,south_america
2,events,healthcare,unemployed,australia
3,paid_ads,retail,NA,australia
4,referral,education,self_employed,europe
...,...,...,...,...
1457,referral,manufacturing,self_employed,north_america
1458,referral,technology,student,europe
1459,paid_ads,technology,student,north_america
1460,referral,NA,self_employed,north_america


In [10]:
df[categorical].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

### Question 1. Mode for Industry

In [11]:
df.industry.mode()

0    retail
Name: industry, dtype: object

Answer 1: retail

### Question 2. Biggest correlation

In [12]:
# Calculating correlation between numerical values
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


Answer 2: annual_income and interaction_count (0.027036)

### Split data

In [13]:
# Split your data in train/val/test sets with 60%/20%/20% distribution.
# Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [14]:
# Reseting the index of datasets
df_val = df_val.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [15]:
# Defining what values we need to predict
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [16]:
# Make sure that the target value y is not in your dataframe.
# Deleting the value to predict to avoid using it in the model
del df_train['converted']
del df_val['converted']
del df_test['converted']

### Question 3. Mutual information score

In [17]:
# Importing a function from SciKit learn that implements MI
from sklearn.metrics import mutual_info_score

In [18]:
# Creating the function to learn which ones are more important
def mutual_info_converted_score(series):
    return mutual_info_score(series, y_train)

In [19]:
# Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
mi = df_train[categorical].apply(mutual_info_converted_score)
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

Answer 3: lead_score (0.035396)

### Question 4. Accuracy

In [20]:
# Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
from sklearn.feature_extraction import DictVectorizer

# Train dataset
dv = DictVectorizer(sparse=False)
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# Validation dataset
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [21]:
# Fit the model on the training dataset. 
from sklearn.linear_model import LogisticRegression

# To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [22]:
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
# Predict based on a chosen threashold
y_pred = model.predict_proba(X_val)[:, 1]
converted_decision = (y_pred >= 0.5)

acc_orig = (y_val == converted_decision).mean()
print("Answer 4:", acc_orig.round(2))

Answer 4: 0.7


### Question 5. Feature selection

In [23]:
df_train.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score'],
      dtype='object')

In [24]:
# Setting datasets
no_industry = ['lead_source', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score']
no_employment_status = ['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'location', 'interaction_count', 'lead_score']
no_lead_score = ['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count']

In [25]:
# No industry
dv_industry = DictVectorizer(sparse=False)
dicts_train_industry = df_train[no_industry].to_dict(orient='records')
dicts_val_industry = df_val[no_industry].to_dict(orient='records')

X_train_industry = dv_industry.fit_transform(dicts_train_industry)
X_val_industry = dv_industry.transform(dicts_val_industry)

# Train model
model_industry = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model_industry.fit(X_train_industry, y_train)

# Prediction
y_pred_industry = model_industry.predict_proba(X_val_industry)[:, 1]
converted_industry = (y_pred_industry >= 0.5)

# Accuracy difference
acc_industry = (y_val == converted_industry).mean()
print("Accuracy without industry is", acc_industry)
industry_diff = abs(acc_industry - acc_orig)
print("Difference:", industry_diff)

Accuracy without industry is 0.6996587030716723
Difference: 0.0


In [26]:
# No employment_status
dv_es = DictVectorizer(sparse=False)
dicts_train_es = df_train[no_employment_status].to_dict(orient='records')
dicts_val_es = df_val[no_employment_status].to_dict(orient='records')

X_train_es = dv_es.fit_transform(dicts_train_es)
X_val_es = dv_es.transform(dicts_val_es)

# Train model
model_es = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model_es.fit(X_train_es, y_train)

# Prediction
y_pred_es = model_es.predict_proba(X_val_es)[:, 1]
converted_es = (y_pred_es >= 0.5)

# Accuracy difference
acc_es = (y_val == converted_es).mean()
print("Accuracy without employment status is", acc_es)
es_diff = abs(acc_es - acc_orig)
print("Difference:", es_diff)

Accuracy without employment status is 0.6962457337883959
Difference: 0.0034129692832763903


In [27]:
# No lead_score
dv_ls = DictVectorizer(sparse=False)
dicts_train_ls = df_train[no_lead_score].to_dict(orient='records')
dicts_val_ls = df_val[no_lead_score].to_dict(orient='records')

X_train_ls = dv_ls.fit_transform(dicts_train_ls)
X_val_ls = dv_ls.transform(dicts_val_ls)

# Train model
model_ls = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model_ls.fit(X_train_ls, y_train)

# Prediction
y_pred_ls = model_ls.predict_proba(X_val_ls)[:, 1]
converted_ls = (y_pred_ls >= 0.5)

# Accuracy difference
acc_ls = (y_val == converted_ls).mean()
print("Accuracy without lead score is", acc_ls)
ls_diff = abs(acc_ls - acc_orig)
print("Difference:", ls_diff)

Accuracy without lead score is 0.7064846416382252
Difference: 0.0068259385665528916


In [28]:
print("Answer 5:", min(industry_diff, es_diff, ls_diff))

Answer 5: 0.0


### Question 6. Parameter tuning

In [32]:
# Train models using all the features as in Q4.
# Train dataset
dv = DictVectorizer(sparse=False)
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# Validation dataset
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

# Empty dict to store accuracy for each regularization parameter
accuracy_dict = {}

# Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
for c in [0.01, 0.1, 1, 10, 100]:
    
    # let's train a regularized logistic regression.
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    # Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
    y_pred = model.predict_proba(X_val)[:, 1]
    converted_decision = (y_pred >= 0.5)
    accuracy = (y_val == converted_decision).mean()
    accuracy_dict[c] = round(accuracy, 3)

# Print the dictionary to check the results
for key, value in accuracy_dict.items():
    print(f"{key}: {value}")

0.01: 0.7
0.1: 0.7
1: 0.7
10: 0.7
100: 0.7


Answer 6: 0.01